In [16]:
# import libraries
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,TensorDataset

In [17]:
# import dataset (comes with seaborn)
import seaborn as sns
iris = sns.load_dataset('iris')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [ ]:
# plot the data
iris.plot(marker='o',linestyle='none',figsize=(12,6))
plt.xlabel('Sample number')
plt.ylabel('Value')
plt.show()

In [18]:
# organize the data

# convert from pandas dataframe to tensor
data = torch.tensor( iris[iris.columns[0:4]].values ).float()

# transform species to number
labels = torch.zeros(len(data), dtype=torch.long)
# labels[iris.species=='setosa']   = 0 # don't need!
labels[iris.species=='versicolor'] = 1
labels[iris.species=='virginica']  = 2

# Break the data into batches

In [19]:
# use scikitlearn to split the data
train_data,test_data, train_labels,test_labels = train_test_split(data, labels, test_size=.2)


# then convert them into PyTorch Datasets (note: already converted to tensors)
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)


# finally, translate into dataloader objects
def load_train_data_batches_with_multipliers(batch_multipliers):
    batch_size_base = 2
    for multiplier in batch_multipliers:
        yield DataLoader(train_data, batch_size=batch_size_base**multiplier, shuffle=True, drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0]) # how big should these batches be??

In [20]:
# check sizes of data batches
for batch in load_train_data_batches_with_multipliers([1, 2, 3, 4, 5, 6]):
  print("batch_size =", batch.batch_size)
  for X, y in batch:
    print(X.shape, y.shape)


batch_size = 2
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch.Size([2, 4]) torch.Size([2])
torch

# Construct the model and training plans

In [ ]:
# a function that creates the ANN model

def create_model():
  # model architecture
  ANNiris = nn.Sequential(
      nn.Linear(4,64),   # input layer
      nn.ReLU(),         # activation unit
      nn.Linear(64,64),  # hidden layer
      nn.ReLU(),         # activation unit
      nn.Linear(64,3),   # output units
        )

  # loss function
  lossfun = nn.CrossEntropyLoss()

  # optimizer
  optimizer = torch.optim.SGD(ANNiris.parameters(), lr=.001)

  return ANNiris,lossfun,optimizer

In [ ]:
# train the model

# global parameter
numepochs = 2500

def train_model(ANNiris, lossfun, optimizer, train_loader):

  # initialize accuracies as empties
  trainAcc = []
  testAcc  = []
  losses   = []

  # loop over epochs
  for epochi in range(numepochs):

    # loop over training data batches
    batchAcc  = []
    batchLoss = []
    for X, y in train_loader:

      # forward pass and loss
      yHat = ANNiris(X)
      loss = lossfun(yHat, y)
      
      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # compute training accuracy just for this batch
      batchAcc.append(100*torch.mean((torch.argmax(yHat, axis=1) == y).float()).item())
      batchLoss.append(loss.item())
    # end of batch loop...

    # now that we've trained through the batches, get their average training accuracy
    trainAcc.append(np.mean(batchAcc))
    losses.append(np.mean(batchLoss))

    # test accuracy
    X, y = next(iter(test_loader)) # extract X,y from test dataloader
    predlabels = torch.argmax(ANNiris(X), axis=1)
    testAcc.append(100*torch.mean((predlabels == y).float()).item())
  
  # function output
  return trainAcc, testAcc, losses


# Test it out

In [ ]:
data = {}
for train_loader in load_train_data_batches_with_multipliers(batch_multipliers=[1, 2, 3, 4, 5, 6]):
    ANNiris, lossfun, optimizer = create_model()
    train_accuracy, test_accuracy, losses = train_model(ANNiris, lossfun, optimizer, train_loader)
    data[train_loader.batch_size] = {
        "train_accuracy": train_accuracy,
        "test_accuracy": test_accuracy,
        "losses": losses
    }

In [ ]:
data

In [ ]:
# plot the results
fig, ax = plt.subplots(1,3,figsize=(15,5))


for batch_size, results in data.items():
    ax[0].plot(results["losses"])
ax[0].set_ylabel('Loss')
ax[0].set_xlabel('Epochs')
ax[0].set_title('Losses with minibatches')
ax[0].legend([batch_size for batch_size in data])

for batch_size, results in data.items():
    ax[1].plot(results["train_accuracy"])
ax[1].set_title('Accuracy with minibatches in training data')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy (%)')
ax[1].legend([batch_size for batch_size in data])
ax[1].set_ylim([27,103])

for batch_size, results in data.items():
    print(batch_size, results)
    ax[2].plot(results["test_accuracy"])
ax[2].set_title('Accuracy with minibatches in test data')
ax[2].set_xlabel('Epochs')
ax[2].set_ylabel('Accuracy (%)')
ax[2].legend([batch_size for batch_size in data])
ax[2].set_ylim([27,103])

plt.show()